In [1]:
%load_ext autoreload
%autoreload 2
import my_baselines

In [2]:
with open('train.json') as f:
    data = f.readlines()

import ast
data = [ast.literal_eval(x) for x in data]

In [3]:
# data[0]

In [4]:
import numpy as np
import time

class MF():

    def __init__(self, R, K, alpha, beta, iterations, all_samples=None):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.

        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """

        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.total_iter = 0
        self.training_process = []
        
        # Create a list of training samples
        if all_samples is None:
            self.all_samples = [
                (i, j, self.R[i, j])
                for i in range(self.num_users)
                for j in range(self.num_items)
                if self.R[i, j] > 0
            ]
        else:
            self.all_samples = all_samples

    def train(self):
        start_time = time.time()
        
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # Perform stochastic gradient descent for number of iterations
        for i in range(self.iterations):
            np.random.shuffle(self.all_samples)
            self.samples = self.all_samples[:128]
            self.sgd()
            
            if (i+1) % 50 == 0:
                mse = self.mse()
                self.training_process.append((i, mse))
                cur_time = time.time() - start_time
                print("%dm:%ds Iteration: %d ; error = %.4f" % (int(cur_time/60), int(cur_time%60) ,i+1, mse))

        self.total_iter += self.iterations
    
    def continue_train(self, iterations):
        start_time = time.time()
        for i in range(iterations):
            np.random.shuffle(self.all_samples)
            self.samples = self.all_samples[:128]
            self.sgd()
            
            if (i+1) % 50 == 0:
                mse = self.mse()
                self.training_process.append((self.total_iter+i, mse))
                cur_time = time.time() - start_time
                print("%dm:%ds Iteration: %d ; error = %.4f" % (int(cur_time/60), int(cur_time%60), 
                                                                self.total_iter+i+1, mse))
        
        self.total_iter += iterations

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)

In [5]:
class Mat:
    def __init__(self):
        self.user2index = {}
        self.item2index = {}
        self.n_user = 0
        self.n_item = 0
        
        self.user2item = {}
        self.item2user = {}
        self.all_rating = 0.
        self.all_cnt = 0
        
        self.all_samples = []
        
    def addEntry(self, entry):
        u = entry['reviewerID']
        i = entry['itemID']
        r = entry['rating']
        
        self.all_rating += r
        self.all_cnt += 1
        
        if u not in self.user2index:
            self.user2index[u] = self.n_user
            self.n_user += 1
        if i not in self.item2index:
            self.item2index[i] = self.n_item
            self.n_item += 1
            
        self.all_samples.append((self.user2index[u], self.item2index[i], r))
        
        if u not in self.user2item:
            self.user2item[u] = [(i,r)]
        else:
            self.user2item[u].append((i,r))
            
        if i not in self.item2user:
            self.item2user[i] = [(u,r)]
        else:
            self.item2user[i].append((u,r))
    
    def addEntries(self, entries):
        for e in entries:
            self.addEntry(e)
        self.all_avg = self.all_rating / self.all_cnt
    
    def ratingMat(self):
        self.R = np.zeros([self.n_user, self.n_item])
        for u, value in self.user2item.items():
            for v in value:
                i, r = v
                user_index = self.user2index[u]
                item_index = self.item2index[i]
                self.R[user_index, item_index] = r
                

In [6]:
RA = Mat()
RA.addEntries(data)
RA.ratingMat()

In [7]:
mfa = MF(RA.R, K=50, alpha=0.1, beta=0.01, iterations=1200, all_samples=RA.all_samples)

In [8]:
mfa.train()

0m:15s Iteration: 50 ; error = 490.7980
0m:27s Iteration: 100 ; error = 486.8199
0m:39s Iteration: 150 ; error = 483.0302
0m:51s Iteration: 200 ; error = 479.3657
1m:3s Iteration: 250 ; error = 476.0416
1m:15s Iteration: 300 ; error = 472.7631
1m:27s Iteration: 350 ; error = 469.7094
1m:39s Iteration: 400 ; error = 466.5976
1m:51s Iteration: 450 ; error = 464.1032
2m:2s Iteration: 500 ; error = 461.6582
2m:14s Iteration: 550 ; error = 459.2103
2m:26s Iteration: 600 ; error = 456.6946
2m:38s Iteration: 650 ; error = 454.2882
2m:50s Iteration: 700 ; error = 452.1412
3m:2s Iteration: 750 ; error = 450.0327
3m:14s Iteration: 800 ; error = 447.9879
3m:26s Iteration: 850 ; error = 445.7256
3m:37s Iteration: 900 ; error = 443.6869
3m:50s Iteration: 950 ; error = 441.7105
4m:2s Iteration: 1000 ; error = 439.4789


In [9]:
def predict(u, i):
    if u not in RA.user2index and i not in RA.item2index:
        return RA.all_avg
    if u not in RA.user2index:
        return sum([r for u,r in RA.item2user[i]]) / len(RA.item2user[i])
    if i not in RA.item2index:
        return sum([r for u,r in RA.user2item[u]]) / len(RA.user2item[u])
    
    user_index = RA.user2index[u]
    item_index = RA.item2index[i]
    r = mfa.get_rating(user_index, item_index)
    if r > 5:
        r = 5
    elif r < 0:
        r = 0
    return r

In [10]:
my_baselines.ratingBaseline(predict)

In [15]:
# mfa.alpha = 0.01
# mfa.beta = 0.001

In [16]:
mfa.continue_train(200)

0m:12s Iteration: 1650 ; error = 423.2552


KeyboardInterrupt: 

In [21]:
mfa.alpha = 0.001
mfa.beta = 0.0001

In [22]:
mfa.continue_train(500)

0m:23s Iteration: 4050 ; error = 399.9140
0m:43s Iteration: 4100 ; error = 399.8937
1m:5s Iteration: 4150 ; error = 399.8738
1m:24s Iteration: 4200 ; error = 399.8533
1m:43s Iteration: 4250 ; error = 399.8335
2m:2s Iteration: 4300 ; error = 399.8126
2m:21s Iteration: 4350 ; error = 399.7925
2m:39s Iteration: 4400 ; error = 399.7705
2m:58s Iteration: 4450 ; error = 399.7493
3m:17s Iteration: 4500 ; error = 399.7303


In [23]:
my_baselines.ratingBaseline(predict)